#Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import cv2
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os

#Data

In [ ]:
from PIL import Image
def load_hazy_dataset(directory, image_size=(256, 256)):
    print(directory + ': ')
    image_files = sorted([f for f in os.listdir(directory) if f.endswith('.png') or
                                                        f.endswith('.jpg') or
                                                        f.endswith('.jpeg')])
    length = len(image_files)
    images = np.empty((length, image_size[0], image_size[1], 3), dtype='float32')

    for i, filename in enumerate(image_files):
        image_path = os.path.join(directory, filename)
        image = Image.open(image_path)
        image = image.resize(image_size)
        images[i] = np.array(image) / 255.0
        print("Loaded image ", i)

    return images

In [ ]:
from PIL import Image
def load_clear_dataset(directory, image_size=(256, 256)):
    print(directory + ': ')
    image_files = sorted([f for f in os.listdir(directory) if f.endswith('.png') or
                                                        f.endswith('.jpg') or
                                                        f.endswith('.jpeg')])
    print(len(image_files))
    images = []

    for i, filename in enumerate(image_files):
        image_path = os.path.join(directory, filename)
        image = Image.open(image_path)
        image = image.resize(image_size)
        for j in range(10):
            images.append(np.array(image) / 255.0)
        print("Loaded image ", i)

    return np.array(images)

In [ ]:
hazy_folder_path = '/content/drive/Othercomputers/My MacBook Pro/Reside/hazy'
hazy_images = load_hazy_dataset(hazy_folder_path)
print(hazy_images.shape)
print(hazy_images[0])

clear_folder_path = '/content/drive/Othercomputers/My MacBook Pro/Reside/clear'
clear_images = load_clear_dataset(clear_folder_path)
print(clear_images.shape)
print(clear_images[0])

# Considering the size of the dataset, data augmentation is probably unneccessary
# and would take a lot of computing resources

Streaming output truncated to the last 5000 lines.
Loaded image  10491
Loaded image  10492
Loaded image  10493
Loaded image  10494
Loaded image  10495
Loaded image  10496
Loaded image  10497
Loaded image  10498
Loaded image  10499
Loaded image  10500
Loaded image  10501
Loaded image  10502
Loaded image  10503
Loaded image  10504
Loaded image  10505
Loaded image  10506
Loaded image  10507
Loaded image  10508
Loaded image  10509
Loaded image  10510
Loaded image  10511
Loaded image  10512
Loaded image  10513
Loaded image  10514
Loaded image  10515
Loaded image  10516
Loaded image  10517
Loaded image  10518
Loaded image  10519
Loaded image  10520
Loaded image  10521
Loaded image  10522
Loaded image  10523
Loaded image  10524
Loaded image  10525
Loaded image  10526
Loaded image  10527
Loaded image  10528
Loaded image  10529
Loaded image  10530
Loaded image  10531
Loaded image  10532
Loaded image  10533
Loaded image  10534
Loaded image  10535
Loaded image  10536
Loaded image  10537
Loaded im

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(hazy_images, clear_images, test_size=0.2, random_state=42, shuffle=True)

#Model Training

In [ ]:
# hyperparameters
NUM_EPOCHS = 20
BATCH_SIZE = 32
INPUT_SHAPE = (256, 256, 3)
TRIAL = 0   # MODIFY THIS BEFORE EVERY RUN
MODEL_SAVE_PATH = f'/content/drive/MyDrive/Synopsys 2024-2025 SIV/Firefighters/Saved Models/dehazer_trial_{TRIAL}.keras'
HISTORY_SAVE_PATH = f'/content/drive/MyDrive/Synopsys 2024-2025 SIV/Firefighters/Saved Models/dehazer_history_trial{TRIAL}.pkl'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_dehazing_model():
    model = keras.Sequential([
        layers.Input(shape=INPUT_SHAPE),

        # Convolutional Layers (Feature Extraction)
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),

        # Convolutional Transpose Layers (Upsampling)
        layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same'),
        layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same'),

        # Output Layer (Clear Image)
        layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same') #sigmoid outputs values between 0 and 1.
    ])
    return model

model = build_dehazing_model()
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 256, 256, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 256, 256, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 256, 256, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 256, 256, 64)        │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 256, 256, 32)        │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 256, 256, 3)         │             867 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 186,371 (728.01 KB)

 Trainable params: 186,371 (728.01 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

checkpoint_callback = ModelCheckpoint(
    filepath=MODEL_SAVE_PATH,
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,
                      y_train,
                      epochs=NUM_EPOCHS,
                      batch_size=BATCH_SIZE,
                      validation_split=0.2,
                      validation_batch_size=BATCH_SIZE,
                      callbacks=[early_stopping_callback, checkpoint_callback])

In [ ]:
model.save(MODEL_SAVE_PATH)

In [ ]:
import pickle
with open(HISTORY_SAVE_PATH, 'wb') as file:
  pickle.dump(history.history, file)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
# Plotting the history
plt.figure(figsize=(10,6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("training and validation accuracy")
plt.xlabel("epoch")
plt.ylabel('accuracy')
plt.legend()
plt.grid(True)
plt.show()